<center> <h2 style="background-color:Orange; color:white" ><br>Teaching Computers to describe pictures<br></h2></center>


**`Image Captioning`**<br> 
* Image Captioning is process of generating textual description of an image
* It uses both Natural Language Processing and Computer Vision to generate captions


**`Table of Contents`** :
1. Introduction
2. Motive
3. Prerequisites
4. Data Collection
5. Interpreting data
6. Data Cleaning
7. Loading the training set
8. Data Preprocessing — Images
9. Data Preprocessing — Captions
10. Data Preparation using Generator Function
11. Word Embeddings
12. Model Architecture
13. Inference
14. Evaluation
15. Conclusion and Future work
16. References



<center> <h3 style="background-color:red; color:white" ><br>1 - Introduction<br></h3></center>


`What is in this image??`
<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.BPMGsi39UsNLDf7-qBZwmwHaC7%26pid%3DApi&f=1"/></center>

Well some might say “A white dog in a grassy area with collar belt”, some may say “White dog with brown spots and collar belt” and yet some others might say “A dog on grass, eating grass and collar belt” maybe

Dont be happy man : Definitely all of these captions are relevant for this image and there may be many others also<br>
Point I want to make is, it’s so easy for human beings to just have a glance at a picture and describe it in an appropriate language. Even a 5 year old could do this with utmost ease now in 2023 yes.

**`Problem Statement`** <br>
`Can you write a computer program that takes an image as input and produces a relevant caption as output for given image?` <br>
With advent of Deep Learning this problem can be solved very easily if one have required dataset.<br>
`Andrej Karapathy` well researched this problem in his PhD thesis at Stanford, who is also now `Director of AI at Tesla`

### AIM
Aim is to explain (in layman term possible) that ""How Deep Learning can be used to solve this problem"" of generating a caption for a given image, hence name **`Image Captioning`** <br>
[Caption Bot](https://captionbot.net/home) ==> state-of-the-art system created by Microsoft


<center> <h3 style="background-color:red; color:white" ><br>2 - Motive<br></h3></center>


Trying to connect this problem to some real world scenarios [Click](https://mobidev.biz/blog/exploring-deep-learning-image-captioning)<br> 
* `Image tagging for ecommerce, Photo-Sharing Seervices and Online Catalogs`
* `Automatic image Annotations for blind peoples`
* `CCTV Cameras` are everywhere today, but along with viewing world, we can also generate relevant captions, then we can raise alarms as soon as there is some malicious activity going on somewhere(malls, roads). This could probably help reduce some crime and/or accidents
* `Automatic Captioning` make a Image Search as good as Google Search here every image could be first converted into a caption and then search can be performed based on caption


<center> <h3 style="background-color:red; color:white" ><br>3 - Prerequisites<br></h3></center>

Know basic Deep Learning concepts before diving in: 
1. Multi-layered Perceptrons
2. Convolution Neural Networks(CNN)
3. Recurrent Neural Networks (RNN)
4. Transfer Learning
5. Gradient Descent
6. Backpropagation
7. Overfitting
8. Probability
9. Text Processing
10. Python (syntax and data structures) 
11. Keras library

**`Encoder`**<br>
Convolutional Neural Network(CNN) can be thought of as an encoder. Input image is given to CNN to extract features. Last hidden state of CNN is connected to Decoder

**`Decoder`**<br>
Decoder is a Recurrent Neural Network(RNN) which does language modelling up to word level. First time step receives encoded output from encoder and also <START> vector

[Understanding Encoder and Decoder Link](https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346)


<center> <h3 style="background-color:red; color:white" ><br>4 - Data Collection<br></h3></center>

Plethora of open source datasets are available for this problem like `Flickr_8k_dataset`(containing8k images), `Flickr_30k_dataset`(containing 30k images), `MS_COCO_dataset`(containing 180k images) more many. <br>
For now I am using `Flickr_8k_dataset` ==>[Download  Dataset here](https://forms.illinois.edu/sec/1713398)<br> 
Why I am using only `Flickr_8k_dataset` not other??? <br>
I dont have a good harware in my rescure alash. No worry I will do it like you can scale it. <br>
This dataset `contains appx8000_images :: 1.1GB apx` `each with 5 captions` ==> (as we have already seen in the Introduction section that an image can have multiple captions, all being relevant simultaneously)<br>

* Dataset will be in form `[image → captions]`
* Dataset consists of input images and their corresponding output captions
<br>

These images are divided into two branches or parts(bifurcated) as follows:
1. `Test Set` — 1000 images
2. `Training Set` — 6000 images
3. `Dev Set` — 1000 images


<center> <h3 style="background-color:red; color:white" ><br>4 - Interpreting Data<br></h3></center>

Downloas data from above provided link 
* Along with images, will also get some text files related to images 
* Dataset will be in form [image → captions]
* Dataset consists of input images and their corresponding output captions

<br>
`captions.txt` ==> contains name of each image along with its 5 captions(0-4)<br>
Reading this file as follows


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
#Image data
path_img = '../input/flickr-8k-images-with-captions/Images'
jpgs = os.listdir(path_img)
#first 10 jpgs
print(f"All jpgs count: {len(jpgs)}\n")
jpgs[:10]

All jpgs count: 8091



['3226254560_2f8ac147ea.jpg',
 '214543992_ce6c0d9f9b.jpg',
 '2366643786_9c9a830db8.jpg',
 '3368819708_0bfa0808f8.jpg',
 '2190227737_6e0bde2623.jpg',
 '2752809449_632cd991b3.jpg',
 '3097776588_312932e438.jpg',
 '1206506157_c7956accd5.jpg',
 '1319634306_816f21677f.jpg',
 '2465218087_fca77998c6.jpg']

In [3]:
#loadng doc into memory function
def load_txt(path_txt):
    #open file as read only
    file = open(path_txt, 'r')
    #read all text
    text = file.read()
    #close file
    file.close()
    return text

In [4]:
path_txt = '../input/flickr8k-text/flickr8k.token.txt'
#loading descriptions
text_doc = load_txt(path_txt)

print(text_doc[:500])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin .
1001773457_577c3a7d70.jpg#0	A black dog and a spotted dog are fighting
1001773457_577c3a7


`NOTE` <br>
Every text line contains `<image name>#n <actual caption>`, where 0 ≤ n ≤ 4 <br>
i.e. `name of image`, `caption number`(0 to 4) and `actual caption`

In [5]:
#reading above text file line after line
txt = []
for line in text_doc.split('\n'): #splitting everyline using '/n'
    #spliting_every_line in words like ==>  '1000268201_693b08cb0e.jpg#0', 'A', 'child', 'in', 'a',
    spliting_every_line = line.split() #dtype : list
    #special cases handling
    if len(spliting_every_line) == 0:
        continue
    #imageName_captionNum_of_all_line list to save 
        # 1. all imageName_captionNum, say like this ==> ['1000268201_693b08cb0e.jpg#0']
        # 2. actual captions,say like this ==> 'A child in a pink dress is climbing up a set of stairs in an entry way .'
        # 3. joined 1 and 2 using " " say like this ==> ['1000268201_693b08cb0e.jpg#0', 'A child in a pink dress is climbing up a set of stairs in an entry way .']
    imageName_captionNum_of_all_line = []
    #list indexing for first element of every line ==> this part ['1000268201_693b08cb0e.jpg#0'] of all
    imageName_captionNum_of_all_line.append(spliting_every_line[0]) #acessing all imageName_captionNum_of_all_line
    #joining name of image, caption number with actual caption, like this ==> ['1000268201_693b08cb0e.jpg#0', 'A child in a pink dress is climbing up a set of stairs in an entry way .'] 
    imageName_captionNum_of_all_line.append(" ".join(spliting_every_line[1:]))
    
    #spliting using "#" in name_of_image#caption_number, output :: ['1000268201_693b08cb0e.jpg', '0']
    imgName_num_seprated = imageName_captionNum_of_all_line[0].split('#') #acessing & spliting this part => '1000268201_693b08cb0e.jpg#0' using indexing
    #appending to above list names as txt
    txt.append(imgName_num_seprated + [imageName_captionNum_of_all_line[1].lower()])

############################################    
#making text dataframe out of raw text    
text_df = pd.DataFrame(txt,columns=['image_name','image_index','image_caption'])
#reindexing
text_df_reindexed = text_df.reindex(columns = ['image_index','image_name','image_caption'])
text_df_reindexed_1 = text_df_reindexed[text_df_reindexed.image_caption != '2258277193_586949ec62.jpg.1']
uni_filename = np.unique(text_df_reindexed_1.image_name.values)

In [6]:
#final dataset for text corpus
text_df = text_df_reindexed_1
print(type(text_df),'\n')
text_df.head()

<class 'pandas.core.frame.DataFrame'> 



,image_index,image_name,image_caption
0,0,1000268201_693b08cb0e.jpg,a child in a pink dress is climbing up a set o...
1,1,1000268201_693b08cb0e.jpg,a girl going into a wooden building .
2,2,1000268201_693b08cb0e.jpg,a little girl climbing into a wooden playhouse .
3,3,1000268201_693b08cb0e.jpg,a little girl climbing the stairs to her playh...
4,4,1000268201_693b08cb0e.jpg,a little girl in a pink dress going into a woo...


<center> <h3 style="background-color:orange; color:white" ><br>Alternate & Effective Approach<br></h3></center>

 
Making dict, here: <br>
* key : image_name without extension
* values : all image_caption related with key

In [7]:
#function to make dict : here "key" ==> image_name without .jpg and #n , "value" as all related actual captions to keys
def load_text_doc(text_toke_data):
    mapping = dict()
    for line in text_toke_data.split('\n'): #splitting everyline using '/n'
        #spliting_every_line in words like ==>  '1000268201_693b08cb0e.jpg#0', 'A', 'child', 'in', 'a',
        spliting_every_line_token = line.split() #dtype : list
        #special cases handling
        if len(line) < 2:
            continue
        
        #first token as IMAGE_ID and rest as DESCRIPTION
        image_id, image_desc = spliting_every_line_token[0], spliting_every_line_token[1:]
        #taking '1000268201_693b08cb0e' only removing .jpg
        image_id = image_id.split('.')[0] #output: 1000268201_693b08cb0e
        image_desc = ' '.join(image_desc) #output: A child in a pink dress is climbing up a set of stairs in an entry way .
        
        if image_id not in mapping:
            mapping[image_id] = list()
        
        #storing
        mapping[image_id].append(image_desc)
    return mapping


In [8]:
#dict with key:values pares, key along with all there descriptions
desc_dict = load_text_doc(text_doc)

print(f'Description Dictionary have :{len(desc_dict.keys())}: number of keys')

Description Dictionary have :8092: number of keys


In [9]:
#accessing all desc_dict.keys() and then making it a list to use indexing and show first 10 keys
list(desc_dict.keys())[:10] #first 10 keys

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615',
 '1007320043_627395c3d8',
 '1009434119_febe49276a',
 '1012212859_01547e3f17',
 '1015118661_980735411b',
 '1015584366_dfcec3c85a']

In [10]:
#number of descriptions for key ==> 1000268201_693b08cb0e
len(desc_dict['1000268201_693b08cb0e'])

5

There are total 5 description for `key:1000268201_693b08cb0e`

In [11]:
#descriptions for key ==> 1000268201_693b08cb0e
desc_dict['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [12]:
#description for key ==> 1015584366_dfcec3c85a
desc_dict['1015584366_dfcec3c85a']

['A black dog leaps over a log .',
 'A grey dog is leaping over a fallen tree .',
 'A large black dog leaps a fallen log .',
 'A mottled black and grey dog in a blue collar jumping over a fallen tree .',
 'The black dog jumped the tree stump .']

`Observation:` <br>
Now we have image_name without extension i.e(.jpg) along with all associated description to it <br>



---
---

In [13]:
import string 

#all punctuation  example 
string.punctuation 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
#removing punctuation dummy
txt = 'A boy in a green shirt is looking down at many inflatable boats .'
mytable = txt.maketrans(" ", " ", string.punctuation)
print(txt.translate(mytable))

A boy in a green shirt is looking down at many inflatable boats 


---
---


## Removing any punctuation and 'a' and 's' from `image_caption`

In [15]:
def clean_image_captions(description_dic):
    
    for key, image_captions in desc_dict.items():
        
        for element_in_image_caption in range(len(image_captions)):
            
            #accessing every element(token) of each image_caption related to key
            image_captions_elements = image_captions[element_in_image_caption]
            #putting every element(token) in seperate list
            image_captions_elements_list = image_captions_elements.split()
            #all in lower case
            image_captions_elements_list_lower_cased = [element.lower() for element in image_captions_elements_list]
            #removing punctuations if exist from each token
            image_captions_elements_punctuations_removed = [element.translate(str.maketrans('', '', string.punctuation)) for element in image_captions_elements_list_lower_cased]
            #removing elements/tokens/word if it's length is less then 1 ==> focus is to remove "s" & "a"
            image_captions_elements_word_cleaned = [element for element in image_captions_elements_punctuations_removed if len(element) > 1]
            
            #removing elements/tokens/word if they are alphanumeric ::: testing here can update later
            image_captions_elements_alphanum_removed = [element for element in image_captions_elements_word_cleaned if element.isalpha()]
            
            #storing final usefull image_captions as strings
            image_captions[element_in_image_caption] = ' '.join(image_captions_elements_alphanum_removed)

##################################
#calling function for work
clean_image_captions(desc_dict)

**`Observation --`** `single letter are removed along with punctuations`

In [16]:
#cleared descriptions for key ==> 1000268201_693b08cb0e
desc_dict['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [17]:
#cleared description for key ==> 1015584366_dfcec3c85a
desc_dict['1015584366_dfcec3c85a']

['black dog leaps over log',
 'grey dog is leaping over fallen tree',
 'large black dog leaps fallen log',
 'mottled black and grey dog in blue collar jumping over fallen tree',
 'the black dog jumped the tree stump']

### Saving processed image_caption text

In [18]:
# function to save processed desc_dict to a new file, one per line
def saving_processed_desc_dict(descriptions_dict, filename):
    lines = list()
    for key, desc_list in desc_dict.items():
        for desc in desc_dict:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

###########################
saving_processed_desc_dict(desc_dict, 'desc_dict_with_processed_image_caption.txt')


### Making img_captions_vocabulary

In [19]:
#function to make a set of vocab out of image_captions
def img_captions_vocabulary(descriptions):
    
    #set to store vocab which we will get out of image_captions of each key
    image_captions_vocabulary_set = set()
    
    for key in descriptions.keys():
        [image_captions_vocabulary_set.update(each_image_captions.split()) for each_image_captions in descriptions[key]]
    
    return image_captions_vocabulary_set

###########################################
#storing vocabulary in variable
image_captions_vocabulary = img_captions_vocabulary(desc_dict) #Result:: {'saddled', 'derby', 'facedown', 'bullbranded', 'obsured', 'cube'} 
print(f'Final image_captions_vocabulary Size: {len(image_captions_vocabulary)}')

Final image_captions_vocabulary Size: 8763


## Loading training file
Here one will get image name with `.jpg` extension, each seprated with `\n` <br>
All images name have number in them like ==> `'2513260012_03d33305cf.jpg\n2903617548_d3e38d7f88.jpg`

In [20]:
#loadng doc into memory function :: making this function again just to make things more clear
def load_txt(path_txt):
    
    #open file as read only
    file = open(path_txt, 'r')
    #read all text
    text = file.read()
    #close file
    file.close()
    return text

#######################################
#loading train data
train_path = '../input/flickr8k-text/'
train_data = load_txt(train_path+'flickr_8k.trainImages.txt')
print(train_data[:200])

2513260012_03d33305cf.jpg
2903617548_d3e38d7f88.jpg
3338291921_fe7ae0c8f8.jpg
488416045_1c6d903fe0.jpg
2644326817_8f45080b87.jpg
218342358_1755a9cce1.jpg
2501968935_02f2cd8079.jpg
2699342860_5288e203e


In [21]:
#if you want to see new line statement by yourself
###train_data

Actual values are seprated using `\n` like this : `'2513260012_03d33305cf.jpg\n2903617548_d3e38d7f88.jpg\n3338291921_fe7ae0c8f8.jpg\n`

One have to split using `\n` to seprate images txt

### Preprocessing Loaded Training File

In [22]:
#function to load and extract train_data image_id only by removing `.jpg` extension and removing empty strings
def load_process_train_data(train_txt_file):
    train_data_final = list()

    #using above function to load train_data
    doc = load_txt(path_txt=train_txt_file)
    
    #spliting image names using `\n` and then accessing every image name
    for every_img_desc in doc.split('\n'):
        
        #to remove any empty string
        if len(every_img_desc) < 1:
            continue
        
        #removing `.jpg` extension or getting image id
        image_id = every_img_desc.split('.')[0]
        train_data_final.append(image_id)
    
    return set(train_data_final)

############################
processed_train_data = load_process_train_data(train_txt_file=train_path+'flickr_8k.trainImages.txt') #dtype :: set
print(f'Total image_id in train_dataset are : {len(processed_train_data)}\n\n')


import random
#see any 5 image_id of train set
print(f"5 Randome train_id's: \n{random.sample(processed_train_data,5)}")

Total image_id in train_dataset are : 6000


5 Randome train_id's: 
['3124549928_10904a5a83', '3138562460_44227a35cf', '265528702_8653eab9fa', '378453580_21d688748e', '3265209567_b3b9c8e0fe']


### Loading and reading image files from Images caption file
`using glob module to retrieve files/pathnames matching a specified pattern `.jpg``

In [23]:
#using glob module to retrieve files/pathnames matching a specified pattern `.jpg`
import glob

#below path contains all images directory
images_data_path = '../input/flickr-8k-images-with-captions/Images/'
#create a list of all image names inside directory Images
every_image_full_path = glob.glob(images_data_path + '*.jpg')

type(every_image_full_path)

list

### All images in train data file
Loading train files

In [24]:
train_images_all = set(open(train_path+'flickr_8k.trainImages.txt', 'r').read().strip().split('\n'))

train_images_after_check = []
for particular_image_full_path in every_image_full_path:
    
    if particular_image_full_path[len(images_data_path):] in train_images_all:
        train_images_after_check.append(particular_image_full_path)
        
#first 3 images with there path :: list of all train images
train_images_after_check[:3]

['../input/flickr-8k-images-with-captions/Images/3226254560_2f8ac147ea.jpg',
 '../input/flickr-8k-images-with-captions/Images/214543992_ce6c0d9f9b.jpg',
 '../input/flickr-8k-images-with-captions/Images/3368819708_0bfa0808f8.jpg']

### All images in test data file
Loading test files

In [25]:
test_images_all = set(open(train_path+'flickr_8k.testImages.txt', 'r').read().strip().split('\n'))

test_images_after_check = []
for particular_image_full_path in every_image_full_path:
    
    if particular_image_full_path[len(images_data_path):] in test_images_all:
        test_images_after_check.append(particular_image_full_path)
        
#first 3 images with there path :: list of all test images
test_images_after_check[:3]

['../input/flickr-8k-images-with-captions/Images/280706862_14c30d734a.jpg',
 '../input/flickr-8k-images-with-captions/Images/929679367_ff8c7df2ee.jpg',
 '../input/flickr-8k-images-with-captions/Images/1317292658_ba29330a0b.jpg']

In [26]:
path_txt = '../input/flickr8k-text/flickr8k.token.txt'

#loadng doc into memory function :: making this function again just to make things more clear
def load_txt(path_txt):
    #open file as read only
    file = open(path_txt, 'r')
    #read all text
    text = file.read()
    #close file
    file.close()
    return text

#loading clear description into memory
def load_clean_descriptions(path_txt, dataset):
    doc = load_txt(path_txt)
    
    descriptions = dict()
    for line in doc.split('\n'):
        #spliting lines by white space
        tokens = line.split()
        #split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        
        if image_id in dataset:
            
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            #wraping description in token
            desc = 'start ' + ' '.join(image_desc) + ' end'
            #store
            descriptions[image_id].append(desc)
    return descriptions

##using function load_clean_descriptions
train_descriptions = load_clean_descriptions('./desc_dict_with_processed_image_caption.txt', processed_train_data)
print(len(train_descriptions))


6000


In [27]:
#preprocessing image paths
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299,299))
    x = image.img_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    return(x)


In [28]:
for i in range(8):
    print(i)
    for j in range(8):
        print(i)
        print(j)
    

0
0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
7
1
1
0
1
1
1
2
1
3
1
4
1
5
1
6
1
7
2
2
0
2
1
2
2
2
3
2
4
2
5
2
6
2
7
3
3
0
3
1
3
2
3
3
3
4
3
5
3
6
3
7
4
4
0
4
1
4
2
4
3
4
4
4
5
4
6
4
7
5
5
0
5
1
5
2
5
3
5
4
5
5
5
6
5
7
6
6
0
6
1
6
2
6
3
6
4
6
5
6
6
6
7
7
7
0
7
1
7
2
7
3
7
4
7
5
7
6
7
7
